In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../cruft/df.csv', nrows=1000)

In [5]:
df['URL'].tolist()

['https://imgix.gizmodo.com.au/content/uploads/sites/2/2015/04/teslahack.png?ar=16%3A9&auto=format&fit=crop&q=65&w=800',
 'https://img.etoday.co.kr/pto_db/2018/11/600/20181115103718_1271208_903_706.JPG',
 'https://i.ytimg.com/vi/BCnQOia9S_I/mqdefault.jpg',
 'https://i2.wp.com/4monimo.com/wp-content/uploads/2018/03/9df5dbd4030e5963dcac4fda7468a3e5.png?resize=150%2C150',
 'http://www.e-vesti.ru/wp-content/uploads/20180106-syr.jpg',
 'https://static.wixstatic.com/media/3fe6dd_5169456c76574b58b61b4fea4f0a0289~mv2.jpg/v1/fill/w_401,h_268,al_c,q_80,usm_0.66_1.00_0.01/Pharmacie%201.jpg',
 'https://3.bp.blogspot.com/-YVFHdAYQPp8/W0oO_mGCmLI/AAAAAAAAH5g/HdpTgU131Usg3JIOyx9RBWtw_Fkv-3JYgCLcBGAs/w240/Job-Market-Fair-Pati-Juli-2018.jpg',
 'http://www.ilrestodelgargano.it/wp-content/uploads/2018/04/gargano_running_week-415x260.jpg',
 'https://www.paganino.de/out/pictures/generated/product/1/460_460_75/ce22358ec06deddffb247e60c87cc6f1_420221.jpg',
 'https://scalehobby.ru/image/cache/catalog/aviaciya